# 感知机实现实验报告

## 包导入

In [1]:
import numpy as np
import math
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import matplotlib as mlp
mlp.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import axes3d, Axes3D

## 感知机类

In [2]:
class Perceptron(object):
    def __init__(self, eta=0.01, n_iter=20):
        self.eta = eta # 学习率
        self.n_iter = n_iter # 迭代次数
    def fit(self, X, y):
        #先对权重参数初始化，然后对训练集中每一个样本循环，根据感知机算法学习规则对权重进行更新
        print(X.shape[1])
        self.wb = np.zeros(1 + X.shape[1]) #初始化权重。数据集特征维数+1
        self.theta1 =[]
        self.theta2 =[]
        self.errors_ =[] #用于记录每一轮中误分类的样本数
        i = 0
        for _ in range(self.n_iter): #迭代n_iter轮
            i = i + 1
            errors = 0 # 初始化错误数量
            for xi, yi in zip(X, y): # 对于每一对(样本特征, 样本类别)
                update = self.eta * (yi - self.predict(xi)) # 调用了predict()函数, [n*yi]
                self.wb[1:] += update * xi # [w + n*yi*xi]
                self.wb[0] += update # [b + n*yi]
                errors += int(update != 0.0) # 如果update不是0,表明出现了误分类的样本,记录在errors中
                err_sum = 0 # 初始化本轮错误数量
                for xj, yj in zip(X, y): # 对于每一对(样本特征, 样本类别)
                    if (yj - self.predict(xj) != 0): # 如果出现错误
                        err_sum += abs(yj - self.predict(xj)) # 错误计数+1
            self.theta1.append(self.wb[1]) # 记录下本轮theta1
            self.theta2.append(self.wb[2]) # 记录下本轮theta2
            self.errors_.append(errors) # 记录下本轮错误总数
        print(self.wb)
        return self
    def net_input(self, xi): # 输入样本特征,输出其在当前感知机函数的y值
        return np.dot(xi, self.wb[1:]) + self.wb[0] #计算向量点乘
    def predict(self, xi):
        return np.where(self.net_input(xi) <= 0.0, -1, 1) # y值>0, 预测为1, 否则为-1
    def plot_decision_regions(self, X, y, resolution=.02): # 绘制二维图
#         fig = plt.figure() # 新建figure
        colors = ['red', 'blue'] # 颜色列表
        markers = ['o', 'x'] # 点形状列表
        cmap = ListedColormap(colors[:len(np.unique(y))]) # 选取列表中的颜色
    
        x1_max, x1_min = max(X[:, 0]) + 1, min(X[:, 0]) - 1 # 设置x的上下限
        x2_max, x2_min = max(X[:, 1]) + 1, min(X[:, 1]) - 1
        # 根据二维的x描绘出网格点
        xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution), np.arange(x2_min, x2_max, resolution))
        Z = self.predict(np.array([xx1.ravel(), xx2.ravel()]).T) # 预测
        print("Z is: ", Z)
        Z = Z.reshape(xx1.shape) # 统一矩阵形状
        plt.contourf(xx1, xx2, Z, alpha=.4, cmap=cmap) # 根据数据绘制等高面
        plt.xlim(xx1.min(), xx1.max()) # 设置绘图的范围
        plt.ylim(xx2.min(), xx2.max())
        for idx, cl in enumerate(np.unique(y)): # 对于每一个(index, 类别)
            x = X[np.where(y==cl), 0] # 当y为cl类时,第一维度的x取值
            y1 = X[np.where(y==cl), 1] # 当y为cl类时,第二维度的x取值
            plt.scatter(x=x, y=y1, alpha=.8, c=cmap(idx), marker=markers[idx], label=cl) # 绘图
        plt.show()
        plt.savefig('./per.png', format='png')
        
    def painter3D(self, theta1, theta2, loss, XX, YY): # 三维图绘制函数
        fig = plt.figure()
        ax1 = Axes3D(fig)
        x, y, z = theta1, theta2, loss
        XXY = XX
        lst = []
        xa, ya = np.meshgrid(np.arange(-10, 10, .5), np.arange(-10, 10, .5))
        sum = xa - xa
        for k in range(0, XXY.shape[0], 1):
            sum += abs(YY[k] - (xa * XXY[k][0] + ya * XXY[k][1]))
        ax1.plot_surface(xa, ya, sum, rstride=1, cstride=1, cmap=plt.cm.Greys, alpha=0.7)
        ax1.plot_wireframe(x, y, z, rstride=5, cstride=5)
        ax1.set_xlabel('theta1')
        ax1.set_ylabel('theta2')
        ax1.set_zlabel('loss')
        plt.show()
        plt.savefig('./per.sgi.png', format='png')

## 主函数

In [3]:
def main(): # 主函数
    iris = load_iris()
    X = iris.data[:100, [0, 1]] # 取鸢尾花数据集前100行,前两列
    y = iris.target[:100] # 取对应的分类标签向量
    y = np.where(y==1, 1, -1) # 将对应的分类标签标准化
    # 分割测试集和训练集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    ppn = Perceptron(eta=0.1, n_iter=19) # 新建感知机
    rst = ppn.fit(X_train, y_train) # 返回训练后的感知机
    ppn.plot_decision_regions(X_test, y_test) # 绘图
    # 绘制3D图
    ppn.painter3D(np.array(rst.theta1).reshape(len(rst.theta1), 1), np.array(rst.theta2).reshape(len(rst.theta2), 1), np.array(rst.errors_).reshape(len(rst.errors_), 1),X,y) 
if __name__ == '__main__':
    main()

2
[-1.8   5.96 -9.08]
Z is:  [1 1 1 ... 1 1 1]


## 运行结果

绘制出的图片如二维散点图以及三维图

![二维散点图](./per.png)

![三维图](./a.png)

## 存在问题

由于原文中的代码是在python2条件下运行的,所以包括print函数在内的很多地方都需要修改.这其中也包括原文中plot_decision_regions函数中的:

```python
plt.scatter(x=X[y==cl, 0], y=X[y==cl, 1], alpha=.8, c=cmap(idx), label=cl)
```

由于在python3的环境中不支持使用boolean index,因此会出现以下错误:boolean index did not match indexed array along dimension 0

正确的方法是将boolean index改为使用numpy中的where函数进行索引:

```python
x = X[np.where(y==cl), 0] # 当y为cl类时,第一维度的x取值
y1 = X[np.where(y==cl), 1] # 当y为cl类时,第二维度的x取值
plt.scatter(x=x, y=y1, alpha=.8, c=cmap(idx), marker=markers[idx], label=cl) # 绘图
```

除此之外,在plot_decision_regions函数中还存在一个疏漏,在绘图前需要调用pyplot中的figure函数新建figure,否则就会出现'NoneType' object has no attribute 'sqrt'的错误

最后还有一个问题在于,在绘制3D图形的时候同样也还可以使用对应theta1, theta2在预测时的错误数作为loss来绘制z轴,具体代码如下:

```python
 def painter3D(self, theta1, theta2, loss, XX, YY): # 三维图绘制函数
        fig = plt.figure()
        ax1 = Axes3D(fig)
        x, y, z = theta1, theta2, loss
        XXY = XX
        lst = []
        xa, ya = np.meshgrid(np.arange(-10, 10, .5), np.arange(-10, 10, .5))
        sum = xa - xa
        for k in range(0, XXY.shape[0], 1):
            array1 = xa * XXY[k][0] + ya * XXY[k][1]
            array1 = np.where(array1 <= 0, -1, 1)
            sum += abs(YY[k] - array1) / 2
        ax1.plot_surface(xa, ya, sum, rstride=1, cstride=1, cmap=plt.cm.Greys, alpha=0.7)
        ax1.plot_wireframe(x, y, z, rstride=5, cstride=5)
        ax1.set_xlabel('theta1')
        ax1.set_ylabel('theta2')
        ax1.set_zlabel('loss')
        plt.show()
        plt.savefig('./per.sgi.png', format='png')
```

通过上述代码绘制出的图像如3D图

![3D图](./per1.sgi.png)

其中z轴表示在对应的theta1以及theta2下分类错误的样本数量,我认为通过这个能够更直观的反映预测时loss的大小